# NTV - JSON Schema extension to NTV data
This Notebook contains the python implementation of the examples presented in the document (examples 1 to 6) as well as more complete examples (examples 7, 8 and 9).
Functions `ntv_validate` and `ntv_validate_opt2` transpose the `validate` function of the `jsonschema` module to NTV data (file [ntv_schema.py](https://github.com/loco-philippe/NTV/blob/main/RFC/ntv_schema.py))

In [1]:
import pathlib
import json_ntv
from json_ntv.namespace import from_file
from RFC.ntv_schema import ntv_validate, ntv_validate_opt2
from json_ntv import Ntv
from jsonschema import validate

## Example 1 - NTV structure

In [2]:
instance = {'family': 'doe', 'childrens age': [15, 24, 12] }
ntv_instance = Ntv.obj(instance)

for ntv in ntv_instance.tree:
    print (ntv.__class__.__name__, ', ', ntv.name, ', ', ntv.type_str, ', ', ntv.ntv_value, ', ', ntv.pointer())

NtvList ,   ,   ,  [{"family": "doe"}, {"childrens age": [15, 24, 12]}] ,  
NtvSingle ,  family ,  json ,  doe ,  /family
NtvList ,  childrens age ,   ,  [15, 24, 12] ,  /childrens age
NtvSingle ,   ,  json ,  15 ,  /childrens age/0
NtvSingle ,   ,  json ,  24 ,  /childrens age/1
NtvSingle ,   ,  json ,  12 ,  /childrens age/2


## Example 2 - NTV instance

In [3]:
instances = [ {'number1': [10, 20] },
              {'number2': {'val1': 10, 'val2': 20}},
              {'number3': [10, {'val2': 20}]}]

schema = { 'properties': {'1': {'minimum': 15}},
           'items': {'maximum': 30},
           'prefixItems': [{'maximum': 15}]}

for instance in instances:
    print ('\n', instance)
    print(ntv_validate(instance, schema, mode=2))


 {'number1': [10, 20]}
     number1/1 - valueNtv  :  20 {'minimum': 15}
     number1/0 - valueNtv  :  10 {'maximum': 30}
     number1/1 - valueNtv  :  20 {'maximum': 30}
     number1/0 - valueNtv  :  10 {'maximum': 15}
True

 {'number2': {'val1': 10, 'val2': 20}}
     number2/val2 - valueNtv  :  20 {'minimum': 15}
     number2/val1 - valueNtv  :  10 {'maximum': 30}
     number2/val2 - valueNtv  :  20 {'maximum': 30}
     number2/val1 - valueNtv  :  10 {'maximum': 15}
True

 {'number3': [10, {'val2': 20}]}
     number3/1 - valueNtv  :  20 {'minimum': 15}
     number3/0 - valueNtv  :  10 {'maximum': 30}
     number3/1 - valueNtv  :  20 {'maximum': 30}
     number3/0 - valueNtv  :  10 {'maximum': 15}
True


## Example 3  - NTVtype and NTVname

In [4]:
instances = [ {'location': 'paris', 'dating:date': '2023-10-01'},
              {'location': 'paris', 'dating:year': 2023} ]

schema = { 'properties': {
                'dating': {'typeNTV': {'enum': ['year', 'date', 'datetime']}}},
           'items': {
                'nameNTV': {'maxLength': 10}}}

for instance in instances:
    print ('\n', instance)
    print(ntv_validate(instance, schema, mode=2))


 {'location': 'paris', 'dating:date': '2023-10-01'}
     /dating:date - valueNtv  :  2023-10-01 {'typeNTV': {'enum': ['year', 'date', 'datetime']}}
     /location - valueNtv  :  paris {'nameNTV': {'maxLength': 10}}
     /dating:date - valueNtv  :  2023-10-01 {'nameNTV': {'maxLength': 10}}
True

 {'location': 'paris', 'dating:year': 2023}
     /dating:year - valueNtv  :  2023 {'typeNTV': {'enum': ['year', 'date', 'datetime']}}
     /location - valueNtv  :  paris {'nameNTV': {'maxLength': 10}}
     /dating:year - valueNtv  :  2023 {'nameNTV': {'maxLength': 10}}
True


## Example 4  - Extension NTV Schema - option 1

In [5]:
instances = [ {'location': 'paris', 'dating:date': '2023-10-01'},
              {'location': 'paris', 'dating:year': 2023} ]

schema = { 'dating': {
                ':typeNtv': {':enum': ['year', 'date', 'datetime'] }},
           ':items': {
                ':nameNtv': {':maxLength': 10}} }

for instance in instances:
    print ('\n', instance)
    print(ntv_validate_opt2(instance, schema, mode=2))


 {'location': 'paris', 'dating:date': '2023-10-01'}
     /dating:date - typeNtv  :  date {'enum': ['year', 'date', 'datetime']}
     /location - nameNtv  :  location {'maxLength': 10}
     /dating:date - nameNtv  :  dating {'maxLength': 10}
True

 {'location': 'paris', 'dating:year': 2023}
     /dating:year - typeNtv  :  year {'enum': ['year', 'date', 'datetime']}
     /location - nameNtv  :  location {'maxLength': 10}
     /dating:year - nameNtv  :  dating {'maxLength': 10}
True


## Example 5 - Extension JSON Schema and NTV Schema - option 2

In [6]:
instances = [ {'location': 'paris', 'dating:date': '2023-10-01'},
              {'location': 'paris', 'dating:year': 2023}]

schema = { '/dating': {
                'typeNtv': {'enum': ['year', 'date', 'datetime']}},
           'items': {
                'nameNtv': {'maxLength': 10}}}

for instance in instances:
    print ('\n', instance)
    print(ntv_validate(instance, schema, mode=2))


 {'location': 'paris', 'dating:date': '2023-10-01'}
     /dating:date - typeNtv  :  date {'enum': ['year', 'date', 'datetime']}
     /location - nameNtv  :  location {'maxLength': 10}
     /dating:date - nameNtv  :  dating {'maxLength': 10}
True

 {'location': 'paris', 'dating:year': 2023}
     /dating:year - typeNtv  :  year {'enum': ['year', 'date', 'datetime']}
     /location - nameNtv  :  location {'maxLength': 10}
     /dating:year - nameNtv  :  dating {'maxLength': 10}
True


## Example 6 - OpenAPI Schema

In [7]:
import pathlib
import json_ntv

file = pathlib.Path(json_ntv.__file__).parent.parent / "RFC" / "NTV_openAPI_namespace.ini"
from_file(file, '$openAPI.')

In [8]:
# example found in the OpenAPI documentation
example_OAS = {"servers": [
     {"url": "https://{username}.gigantic-server.com:{port}/{basePath}",
      "description": "The production API server",
      "variables": {
        "username": {
          "default": "demo",
          "description": "this value is assigned by the service provider, in this example `gigantic-server.com`"},
        "port": {
          "enum": ["8443", "443"],
          "default": "8443"},
        "basePath": {
          "default": "v2"
        }}}]}

In [9]:
# Example - with NTV types
example_NTV = {
    "example:$openAPI.":
        {"servers.": [
            {":url": "https://{username}.gigantic-server.com:{port}/{basePath}",
             ":description": "The production API server", 
             "variables.": {
                "username": {
                  ":default": "demo",
                  ":description": "this value is assigned by the service provider, in this example `gigantic-server.com`"},
                "port": {
                  ":enum": ["8443", "443"],
                  "default": "8443" },
                "basePath": {
                  ":default": "v2"}}}]}}
schema_NTV = Ntv.obj(example_NTV)
print(schema_NTV['#example:$openAPI./servers./0/variables./username/:default'])
print(schema_NTV['#example/servers./0/variables./username/:default'])
print(schema_NTV['#example/servers./0/variables./username/:default'].ntv_type)

{":$openAPI.servers.variables.default": "demo"}
{":$openAPI.servers.variables.default": "demo"}
$openAPI.servers.variables.default


In [10]:
# Example - with comments
schema_NTV2 = Ntv.obj(
    {"example:$openAPI.servers." : { 
        "server1": {
             ":url": "https://{username}.gigantic-server.com:{port}/{basePath}",
             "prod:description": "The production API server", 
             "variables.": {
                "username": {
                  "user:default": "demo",
                  ":description": "this value is assigned by the service provider, in this example `gigantic-server.com`"},
                "port": {
                  ":enum": ["8443", "443"],
                  "default": "8443" },
                "basePath": {
                  ":default": "v2"}}}}})
print(schema_NTV2['#example:$openAPI.servers./server1/variables./username/user:default'])
print(schema_NTV2['#example/server1/variables./username/user'])

{"user:$openAPI.servers.variables.default": "demo"}
{"user:$openAPI.servers.variables.default": "demo"}


## Example 7 - overview - JSON Schema

In [11]:
data_product = {
    "name": "Widget",
    "price": 10.99,
    "quantity": 5 }

json_schema = {
    "title": "Product",
    "properties": {          
        "name": {
            "type": "string" },
        "price": {
            "type": "number", 
            "minimum": 0 },
        "quantity": {
            "type": "integer", 
            "minimum": 1 } },
    "required": [ "name", "price", "quantity"] }

# example JSON Schema
try:
    validate(data_product, json_schema)
    print("Valid JSON object.")
except Exception as e:
    print("Invalid JSON object:", e)
    
# example NTV Schema option 1
schema_opt1 = {
    "title": "Product",
    "/name": {
        "type": "string" },
    "/price": {
        "type": "number", 
        "minimum": 0 },
    "/quantity": {
        "type": "integer", 
        "minimum": 1 } ,
    "required": [ "/name", "/price", "/quantity"] }
print('NTV schema 1 - valid :', ntv_validate(data_product, schema_opt1))

# example NTV Schema option 2
schema = {
    ":title": "Product - NTVschema",
    "name": {
        ":type": "string" },
    "price": {
        ":type": "number", 
        ":minimum": 0 },
    "quantity": {
        ":type": "integer", 
        ":minimum": 1 },
    ":required": [ "name", "price", "quantity"] }
print('NTV schema 2 - valid :', ntv_validate_opt2(data_product, schema))

Valid JSON object.
NTV schema 1 - valid : True
NTV schema 2 - valid : True


## Example 8 - complete JSON Schema and NTV Schema

In [12]:
json_test = {'family' : 'white', 
             'childrens': [24, 
                           15, 
                           {'walter jr': None}, 
                           {'judith': 21}]}

json_schema = {'properties': {
                  'family':{
                        'maxLength':10,
                        'type': ['string', 'integer']},
                  'childrens':{
                        'maxItems': 4,
                        'items': {
                            'maximum': 25, 
                            'propertyNames': {'maxLength':10},
                            'properties':{
                                'judith': {'maximum': 23}}}}}}

validate(json_test, json_schema)
print('\n', ntv_validate(json_test, json_schema, mode=2))

json_schema2 = {'properties': {
                    'family':{
                        'maxLength':10},
                    'childrens':{
                        'maxItems': 4,
                        'items': {
                            'maximum': 25, 
                            'propertyNames': {'maxLength':10}},
                        '/1': {'minimum':16},
                        '/judith': {'maximum': 20}}}}

validate(json_test, json_schema2)
print('\n', ntv_validate(json_test, json_schema2, mode=1))

     /family - valueNtv  :  white {'maxLength': 10, 'type': ['string', 'integer']}
     /childrens/0 - valueNtv  :  24 {'maximum': 25}
     /childrens/0 - nameNtv  :   {'maxLength': 10}
     /childrens/1 - valueNtv  :  15 {'maximum': 25}
     /childrens/1 - nameNtv  :   {'maxLength': 10}
     /childrens/2 - valueNtv  :  None {'maximum': 25}
     /childrens/2 - nameNtv  :  walter jr {'maxLength': 10}
     /childrens/3 - valueNtv  :  21 {'maximum': 23}
     /childrens/3 - valueNtv  :  21 {'maximum': 25}
     /childrens/3 - nameNtv  :  judith {'maxLength': 10}
     /childrens - valueNtv  :  [24, 15, {"walter jr": null}, {"judith": 21}] {'maxItems': 4}

 True
  validate :  
  validate :  /family
  validate :  /childrens
  validate :  /childrens/0
  validate :  /childrens/1
  validate :  /childrens/2
  validate :  /childrens/3
  validate :  /childrens/1
  error  /childrens/1 - valueNtv  :  15 is not valid with schema :  {'minimum': 16}
  validate :  /childrens/3
  error  /childrens/3 - valu

## Example 9 - nested types - NTV Schema

In [13]:
ntv_test2 = {
      'example family::org.' : {
             'name': 'white',
             'House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets::':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3}}}

ntv_schema3  = {
        'properties': {
            'House.': {
                'properties':{
                    'location':{
                        'typeNtv': {'enum': ['point', 'geojson', 'org.Place.address']}},
                    'arrival': {
                        'typeNtv': {'enum': ['year', 'date', 'datetime']}}}},
            'childrens':{  
                 ':maxItems': 5,
                 'items': {
                     'properties':{
                         'age':{
                             'maximum': 25,
                             'typeNtv': {'enum': ['json', 'int']},
                             }}}},
            'pets':{
               'prefixItems':[
                   {'typeNtv': {'const':'$dog'}}],
               'items':{
                   'typeNtv': {'enum': ['$dog', '$cat', '$mouse']}},
               'properties': {
                   ':$mouse': {'maximum': 10 }}}}}
ntv_schema4  = {       
        '/House.': {
            '/location':{
                'typeNtv': {'enum': ['point', 'geojson', 'org.Place.address']}},
            '/arrival': {
                'typeNtv': {'enum': ['year', 'date', 'datetime']}}},
        '/childrens':{  
             'maxItems': 5,
             'items': {
                 '/age':{
                     'maximum': 25,
                     'typeNtv': {'enum': ['json', 'int']}}}},
        '/pets':{
           '/0':{ 
                'typeNtv': {'const':'$dog'}},
           'items':{
                'typeNtv': {'enum': ['$dog', '$cat', '$mouse']}},                  
            '/:$mouse': {'maximum': 10 }}}
                    
print(ntv_validate(ntv_test2, ntv_schema3, mode=1))
print(ntv_validate(ntv_test2, ntv_schema4, mode=1))

  validate :  example family:org.
  validate :  example family:org./House.
  validate :  example family:org./House./location:point
  validate :  example family:org./House./arrival:year
  validate :  example family:org./childrens:Person.
  validate :  example family:org./childrens:Person./0
  validate :  example family:org./childrens:Person./0
  validate :  example family:org./childrens:Person./1
  validate :  example family:org./childrens:Person./2
  validate :  example family:org./childrens:Person./2/age
  validate :  example family:org./pets
  validate :  example family:org./pets/:$dog
  validate :  example family:org./pets/:$dog
  validate :  example family:org./pets/:$cat
  validate :  example family:org./pets/:$mouse
  validate :  example family:org./pets/:$mouse
True
  validate :  example family:org.
  validate :  example family:org./House.
  validate :  example family:org./House./location:point
  validate :  example family:org./House./arrival:year
  validate :  example family:or